# Unfaithful Medium-sized DAG
## 120521

In [1]:
from causalsage import *

![](2.9.png)

0 = Z1

1 = Z2

2 = Z3

3 = X

4 = W

5 = Y

In [2]:
%display plain

In [3]:
for i in range(6):
    for j in range(6):
        var('beta%d%d' % (i, j))

# Compute Overall Covariance Matrix from DAG

$$W_{ij} = W_{i \gets j}$$

In [4]:
W = matrix([[0, 0, 0, 0, 0, 0], # To Z1
           [0, 0, 0, 0, 0, 0], # To Z2
           [beta02, beta12, 0, 0, 0, 0], # To Z3
           [beta03, 0, beta23, 0, 0, 0], # To X
           [0, 0, 0, beta34, 0, 0], # To W
           [0, beta15, beta25, 0, beta45, 0]]) # To Y

W

[     0      0      0      0      0      0]
[     0      0      0      0      0      0]
[beta02 beta12      0      0      0      0]
[beta03      0 beta23      0      0      0]
[     0      0      0 beta34      0      0]
[     0 beta15 beta25      0 beta45      0]

In [5]:
P = (identity_matrix(len(W.columns())) - W).inverse()
P

[                                                     1                                                      0                                                      0                                                      0                                                      0                                                      0]
[                                                     0                                                      1                                                      0                                                      0                                                      0                                                      0]
[                                                beta02                                                 beta12                                                      1                                                      0                                                      0                                                      0]
[   

In [6]:
Cov_total = P*P.T

[Cov_total[i, i] for i in range(6)]

[1,
 1,
 beta02^2 + beta12^2 + 1,
 beta12^2*beta23^2 + (beta02*beta23 + beta03)^2 + beta23^2 + 1,
 beta12^2*beta23^2*beta34^2 + (beta02*beta23 + beta03)^2*beta34^2 + beta23^2*beta34^2 + beta34^2 + 1,
 beta34^2*beta45^2 + (beta12*beta23*beta34*beta45 + beta12*beta25 + beta15)^2 + ((beta02*beta23 + beta03)*beta34*beta45 + beta02*beta25)^2 + (beta23*beta34*beta45 + beta25)^2 + beta45^2 + 1]

# Compute Partial Covariance Matrix from DAG

Recall that

$$
\begin{aligned}  
	 \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{ZX} \\
     \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{XZ}^{T}
\end{aligned}
$$

So you need: the indices for X and the indices for Z

You can then extract the relevant submatrices and perform the computation.

## Example: Compute Cov(0, 1 | 2)

In [7]:
targets = [0, 1]
condset = [2]

In [8]:
SigmaX = extract_submatrix(targets, targets, Cov_total)

SigmaX

[1 0]
[0 1]

In [9]:
SigmaXZ = extract_submatrix(targets, condset, Cov_total)
SigmaXZ

[beta02]
[beta12]

In [10]:
SigmaZ = extract_submatrix(condset, condset, Cov_total)

SigmaZ

[beta02^2 + beta12^2 + 1]

In [11]:
SigmaXgZ = SigmaX  - SigmaXZ*SigmaZ.inverse()*SigmaXZ.T
SigmaXgZ

[ -beta02^2/(beta02^2 + beta12^2 + 1) + 1 -beta02*beta12/(beta02^2 + beta12^2 + 1)]
[-beta02*beta12/(beta02^2 + beta12^2 + 1)  -beta12^2/(beta02^2 + beta12^2 + 1) + 1]

$$
\begin{aligned}  
	 \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{ZX} \\
     \Sigma_{X \mid Z} &= \Sigma_{X} - \Sigma_{XZ} \Sigma_{ZZ}^{-1} \Sigma_{XZ}^{T}
\end{aligned}
$$

In [12]:
partial_covariance_matrix(targets, condset, Cov_total)

[ -beta02^2/(beta02^2 + beta12^2 + 1) + 1 -beta02*beta12/(beta02^2 + beta12^2 + 1)]
[-beta02*beta12/(beta02^2 + beta12^2 + 1)  -beta12^2/(beta02^2 + beta12^2 + 1) + 1]

## Example: Cov(0, 4 | 3)

This **should be** 0.

In [13]:
targets = [0, 4]
condset = [3]

#partial_covariance_matrix(targets, condset, Cov_total)
partial_covariance_matrix(targets, condset, Cov_total)[0, 1].full_simplify()

0

## Example Cov(1, 2 | 0, 3, 4, 5)

Z2 **should not** be independent of Z3 conditional on the other variables if the distribution is faithful to the DAG.

In [14]:
targets = [1, 2]
condset = [0, 3, 4, 5]

partial_covariance_matrix(targets, condset, Cov_total)[0, 1].full_simplify()

-(beta15*beta25 - beta12)/((beta12^2 + beta15^2 + 1)*beta23^2 + 2*beta12*beta15*beta25 + (beta12^2 + 1)*beta25^2 + beta15^2 + 1)

# Compute Partial Regression Coefficients from DAG

$$\beta_{YX \cdot Z} = \frac{\gamma_{YX \cdot Z}}{\sigma_{X \cdot Z}^{2}}$$

In [18]:
def partial_regression_coefficient(response, predictor, condset, Sigma):
	targets = [response, predictor]

	SigmaX = extract_submatrix(targets, targets, Sigma)
	SigmaXZ = extract_submatrix(targets, condset, Sigma)
	SigmaZ = extract_submatrix(condset, condset, Sigma)
	
	SigmaXgZ = SigmaX  - SigmaXZ*(SigmaZ.inverse())*SigmaXZ.T
	
	return SigmaXgZ[0, 1]/SigmaXgZ[1, 1]

In [21]:
partial_regression_coefficient(0, 1, [2, 3, 4, 5], Cov_total).full_simplify()

-beta02*beta12/(beta02^2 + beta03^2 + 1)